In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-orig/test.csv
/kaggle/input/submission/sample_submission.csv
/kaggle/input/train-dataset/train.csv
/kaggle/input/train-dataset/test.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score as AUC
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, LabelBinarizer

from scipy import stats
import seaborn as sns
from copy import deepcopy

%matplotlib inline


import xgboost as xgb
import pickle
import sys
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix, hstack
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV 

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor 
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.linear_model import Ridge

In [3]:
print('Loadind data from csv files.....')
train_data = pd.read_csv('../input/train-dataset/train.csv')
test_data  = pd.read_csv('../input/test-orig/test.csv')
print('shape of train data :',train_data.shape)
print('shape of test data :',test_data.shape)


test_data['loss'] = np.nan
combined_data = pd.concat([train_data, test_data])
print('Joining complete....')
print('shape of combined data:',combined_data.shape)


categorical_features = list(train_data.select_dtypes(include = ['object']).columns)
print('Total number of categorical features in data are :', len(categorical_features))

no_common = []
for i in categorical_features:
    if train_data[i].nunique() != test_data[i].nunique():
        no_common.append(i)
print('Features that have diff unique values in train and test are :')
print(no_common)

def log_xgboost_eval_mae(pred,dtrain):
    labels = dtrain.get_label()
    a = mean_absolute_error(np.exp(pred),np.exp(labels))
    return 'mae',a

def search_feature(x):
    if x in combined_remaining:
        return np.nan
    return 

#Reference https://www.geeksforgeeks.org/python-pandas-factorize/
#Reference https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe

for i in categorical_features:
    if train_data[i].nunique() != test_data[i].nunique():
        train_unique_set = set(train_data[i].unique())
        test_unique_set  = set(test_data[i].unique())
        remaining_train  = train_unique_set - test_unique_set
        remaining_test   = test_unique_set - train_unique_set
        
        combined_remaining = remaining_train.union(remaining_test)
        
        combined_data[i] = combined_data[i].apply(lambda x: search_feature(x),1)
    combined_data[i] = pd.factorize(combined_data[i].values,sort = True)[0]
    
x_train = combined_data[combined_data['loss'].notnull()]
x_test = combined_data[combined_data['loss'].isnull()]

shift = 200
y_train = np.log(x_train['loss']+shift)
x_train = x_train.drop(['loss','id'],axis = 1)
x_test  = x_test.drop(['loss','id'],axis = 1)

print(x_train.shape)
print(x_test.shape)

Loadind data from csv files.....
shape of train data : (188318, 132)
shape of test data : (125546, 131)
Joining complete....
shape of combined data: (313864, 132)
Total number of categorical features in data are : 116
Features that have diff unique values in train and test are :
['cat90', 'cat92', 'cat96', 'cat99', 'cat101', 'cat102', 'cat103', 'cat105', 'cat106', 'cat109', 'cat110', 'cat113', 'cat114', 'cat116']
(188318, 130)
(125546, 130)


In [4]:
#Combining both x_train and y_train to do the row sampling 
train = pd.concat([x_train,y_train],axis =1)
n_rows = 113008 #60% of data points.

def row_sampling(train,seed):
    sample_df = train.sample(n = n_rows,replace = False, random_state=seed)
    return sample_df

def split_xy(train):
    x = train.drop(['loss'],axis =1)
    y = train.loss
    return x,y


data1 = row_sampling(train,1)
x_train1,y_train1 = split_xy(data1)

data2 = row_sampling(train,2)
x_train2,y_train2 = split_xy(data2)

data3 = row_sampling(train,3)
x_train3,y_train3 = split_xy(data3)

data4 = row_sampling(train,4)
x_train4,y_train4 = split_xy(data4)

data5 = row_sampling(train,5)
x_train5,y_train5 = split_xy(data5)

In [5]:
model_1 = RandomForestRegressor()
model_2 = DecisionTreeRegressor()
model_3 = ElasticNet()
model_4 = Ridge()
model_5 = ExtraTreesRegressor()

## Hyper Parameter Tuning for all Base Models

In [23]:
rf_params = {
    'n_estimators': [90,100,125,150],
    'max_features': [0.2,0.1,0.3],
    'max_depth': [7,8,9,10],
    'min_samples_leaf': [1,2,3,4],
    'min_samples_split': [10, 20, 40]
}
clf1 = GridSearchCV(estimator = model_1,param_grid = rf_params,cv=3)
clf1.fit(x_train1,y_train1)
final_model1 = clf1.best_estimator_
print(final_model1)

dt_params = {
      "min_samples_split": [10, 20, 40],
      "max_depth": [2, 6, 8],
      "min_samples_leaf": [20, 40, 100],
      "max_leaf_nodes": [5, 20, 100],
              }

clf2 = GridSearchCV(estimator = model_2,param_grid = dt_params,cv=3)
clf2.fit(x_train2,y_train2)
final_model2 = clf2.best_estimator_
print(final_model2)


eln_params = {
    'alpha':[1.0,0.1,0.01,0.001,0.005,0.015],
    'l1_ratio':[0.4,0.3,0.2,0.1,0.01,0.05]
}

clf3 = GridSearchCV(estimator = model_3,param_grid = eln_params,cv=3)
clf3.fit(x_train3,y_train3)
final_model3 = clf3.best_estimator_
print(final_model3)

rid_params = {
    'alpha':[1.0,0.1,0.01,0.001,0.005,0.015]
}

clf4 = GridSearchCV(estimator = model_4,param_grid = rid_params,cv=3)
clf4.fit(x_train4,y_train4)
final_model4 = clf3.best_estimator_
print(final_model4)

et_params = {
    'n_estimators': [100,125,150,175],
    'max_features': [0.2,0.1,0.3],
    'max_depth': [7,8,9,10],
    'min_samples_leaf': [1,2,3,4],
}
clf5 = GridSearchCV(estimator = model_5,param_grid = et_params,cv=3)
clf5.fit(x_train5,y_train5)
final_model5 = clf5.best_estimator_
print(final_model5)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=8, max_features=0.2, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)
DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=12,
                      max_features=0.3, max_leaf_nodes=120,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=15, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')
ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.01,
           max_iter=1000, normalize=Fa

## Fitting with best hyper parameters


In [6]:
final_model1 = RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=8, max_features=0.2, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=2,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

final_model2 = DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=12,
                      max_features=0.3, max_leaf_nodes=120,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=15, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

final_model4 = ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.01,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

final_model3 = ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.01,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)


final_model5 = ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=10, max_features=0.3, max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=3,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=150, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)

In [7]:
final_model1.fit(x_train1,y_train1)
final_model2.fit(x_train2,y_train2)
final_model3.fit(x_train3,y_train3)
final_model4.fit(x_train4,y_train4)
final_model5.fit(x_train5,y_train5)

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=10, max_features=0.3, max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=3,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=150, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)

## Making Predictions based on base models.

In [8]:
pred_list = []

pred1 = final_model1.predict(x_train)
pred_list.append(pred1)

pred2 = final_model2.predict(x_train)
pred_list.append(pred2)

pred3 = final_model3.predict(x_train)
pred_list.append(pred3)

pred4 = final_model4.predict(x_train)
pred_list.append(pred4)

pred5 = final_model5.predict(x_train)
pred_list.append(pred4)


## Converting predictions into a dataframe

In [9]:
base_learner_df = pd.DataFrame()
df_cols = ['RF','DT','ELN','RD','ET']
for i,j in enumerate(df_cols):
    base_learner_df[j] = pred_list[i]

In [10]:
x_train_meta = base_learner_df
y_train_meta = train['loss']
print(x_train_meta.shape)
print(y_train_meta.shape)


(188318, 5)
(188318,)


In [20]:
xgb_params = {
    'seed': [0],
    'colsample_bytree': [0.7,0.8,0.6],
    'silent': [1],
    'subsample': [0.5,0.6,0.7],
    'learning_rate': [0.01,0.0001,0.1],
    'objective': ['reg:linear'],
    'max_depth': [3,4,5],
    'num_parallel_tree': [1,2,3],
    'min_child_weight': [1,2,3],
    'eval_metric': ['mae'],
}

model_meta = XGBRegressor()
clf6 = GridSearchCV(estimator = model_meta,param_grid = xgb_params,cv=3)
clf6.fit(x_train_meta,y_train_meta)
final_model6 = clf6.best_estimator_
print(final_model6)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eval_metric='mae',
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints=None, learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=3, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=0,
             num_parallel_tree=3, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=1,
             subsample=0.7, tree_method=None, validate_parameters=False,
             verbosity=None)


In [14]:
final_model6.fit(x_train_meta,y_train_meta)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, eval_metric='mae',
             gamma=0, gpu_id=-1, importance_type='gain',
             interaction_constraints=None, learning_rate=0.1, max_delta_step=0,
             max_depth=5, min_child_weight=3, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=0,
             num_parallel_tree=3, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=1,
             subsample=0.7, tree_method=None, validate_parameters=False,
             verbosity=None)

In [16]:
list1=[]
model_list = [final_model1,final_model2,final_model3,final_model4,final_model5]
for i in range(0,5):
    list1.append(model_list[i].predict(x_test))
    
test_df = pd.DataFrame()
for i,j in enumerate(df_cols):
    test_df[j] = list1[i]

print(test_df.shape)

(125546, 5)


In [18]:
p_test = np.exp(final_model6.predict(test_df)) - 200
sub = pd.read_csv('../input/submission/sample_submission.csv')
sub['loss'] = p_test
sub.to_csv('submissions1.csv', index=False)

## Observations:
* we got a score of 1194.85.